In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install segmentation_models_pytorch 
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 23.2 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16444 sha256=1ac144ff318b302cac87411ffeefc7f57cf2e0140aa558898a8f132d85cdd7a4
  Stored in directory: /root/.cache/pip/wheels/29/16/24/752e89d88d333af39a288421e64d613b5f652918e39ef1f8e3
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=7ed7e22f217c78bb3b6dcaf7040331a053f13660a61a883e25bf709c41650a0f
  Stored in directory: /root/.cache/pip/w

In [ ]:
from collections import namedtuple
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.io as tv
import torchvision.transforms as transf
import torch.utils.data
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from torch.optim import Adam
from tqdm import tqdm
import segmentation_models_pytorch as smp
import torchmetrics
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# For reproducibility
torch.manual_seed(100)
if torch.cuda.is_available():
    torch.cuda.manual_seed(100)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

#################################################################################################################################
# Label nemedtuple
# taken from Dataset offical Github: https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/helpers/labels.py
#################################################################################################################################
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      0 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      0 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      0 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      0 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      0 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      0 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      0 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,      1 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,      2 , 'flat'            , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,      0 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      0 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,      3 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,      4 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,      5 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      0 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      0 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      0 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,      6 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      0 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,      7 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,      8 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,      9 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,     10 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,     11 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,     12 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,     13 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,     14 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,     15 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,     16 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      0 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      0 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,     17 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,     18 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,     19 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , -1 ,      0 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]

#################################################################################################################################
# Define custom Train and Validation set classes
# This assusmes that you have downloaded the Cityscpes dataset, and placed the masks with your training IDs in the meta folder
# Method inspired from: https://github.com/fregu856/deeplabv3/blob/master/datasets.py
#################################################################################################################################
train_dirs = ["jena/", "zurich/", "weimar/", "ulm/", "tubingen/", "stuttgart/",
              "strasbourg/", "monchengladbach/", "krefeld/", "hanover/",
              "hamburg/", "erfurt/", "dusseldorf/", "darmstadt/", "cologne/",
              "bremen/"]
val_dirs = ["frankfurt/", "munster/", "lindau/"]
test_dirs = ["bochum/", "aachen/"]

cityscapes_data_path = '../content/drive/MyDrive/Cityscapes_data'
cityscapes_meta_path = '../content/drive/MyDrive/Cityscapes_data/meta'

class DatasetTrain(torch.utils.data.Dataset):
  def __init__(self, cityscapes_data_path, cityscapes_meta_path):
    self.img_dir = cityscapes_data_path + "/leftImg8bit/train/"
    self.label_dir = cityscapes_meta_path + "/label_imgs/"

    self.examples = []

    for train_dir in train_dirs:
      train_img_dir_path = self.img_dir + train_dir

      file_names = os.listdir(train_img_dir_path)
      for file_name in file_names:
        img_id = file_name.split("_leftImg8bit.png")[0]
        img_path = train_img_dir_path + file_name
        label_img_path = self.label_dir + img_id + ".png"

        example = {}
        example["img_path"] = img_path
        example["label_img_path"] = label_img_path
        example["img_id"] = img_id
        self.examples.append(example)

    self.num_examples = len(self.examples)

  def __len__(self):
    return self.num_examples

  def __getitem__(self, index):
    example = self.examples[index]

    img_path = example["img_path"]
    transform = transf.Resize((256,512), antialias=True)
    img = tv.read_image(img_path)
    img = transform(img)
    img = img.to(torch.float32) / 255.0

    label_img_path = example["label_img_path"]
    label_img = tv.read_image(label_img_path)
    label_img = transform(label_img)
    label_img = label_img.to(torch.float32)

    return (img, label_img)

class DatasetVal(torch.utils.data.Dataset):
  def __init__(self, cityscapes_data_path, cityscapes_meta_path):
    self.img_dir = cityscapes_data_path + "/leftImg8bit/val/"
    self.label_dir = cityscapes_meta_path + "/label_imgs/"

    self.examples = []

    for val_dir in val_dirs:
      val_img_dir_path = self.img_dir + val_dir

      file_names = os.listdir(val_img_dir_path)
      for file_name in file_names:
        img_id = file_name.split("_leftImg8bit.png")[0]
        img_path = val_img_dir_path + file_name
        label_img_path = self.label_dir + img_id + ".png"

        example = {}
        example["img_path"] = img_path
        example["label_img_path"] = label_img_path
        example["img_id"] = img_id
        self.examples.append(example)

    self.num_examples = len(self.examples)

  def __len__(self):
    return self.num_examples

  def __getitem__(self, index):
    example = self.examples[index]

    img_path = example["img_path"]
    
    transform = transf.Resize((256,512), antialias=True)
    img = tv.read_image(img_path)
    img = transform(img)
    img = img.to(torch.float32) / 255.0

    label_img_path = example["label_img_path"]
    label_img = tv.read_image(label_img_path)
    label_img = transform(label_img)
    label_img = label_img.to(torch.float32)

    return (img, label_img)

train_dataset = DatasetTrain(cityscapes_data_path=cityscapes_data_path,
                             cityscapes_meta_path=cityscapes_meta_path)

val_dataset = DatasetVal(cityscapes_data_path=cityscapes_data_path,
                         cityscapes_meta_path=cityscapes_meta_path)

#################################################################################################################################
# Define UNET
#################################################################################################################################
class DoubleConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(DoubleConv, self).__init__()
    self.conv = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
      nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
      )
    
  def forward(self, x):
    return self.conv(x)

class UNet(nn.Module):
  def __init__(self, in_channels, out_channels, inter_channels=[64, 128, 256, 512]):
    super(UNet, self).__init__()
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    # Encoder - Contracting path (down path)
    self.e1 = DoubleConv(in_channels, inter_channels[0])
    self.e2 = DoubleConv(inter_channels[0], inter_channels[1])
    self.e3 = DoubleConv(inter_channels[1], inter_channels[2])
    self.e4 = DoubleConv(inter_channels[2], inter_channels[3])

    # Bottom part of the network
    self.bottom = DoubleConv(inter_channels[-1], inter_channels[-1]*2)

    # Decoder - Expansive path (up path)
    rev_channels = inter_channels[::-1]
    self.d4_T = nn.ConvTranspose2d(rev_channels[0]*2, rev_channels[0], kernel_size=2, stride=2, padding=0)
    self.d4_C = DoubleConv(rev_channels[0]*2, rev_channels[0])
    self.d3_T = nn.ConvTranspose2d(rev_channels[0], rev_channels[1], kernel_size=2, stride=2, padding=0)
    self.d3_C = DoubleConv(rev_channels[0], rev_channels[1])
    self.d2_T = nn.ConvTranspose2d(rev_channels[1], rev_channels[2], kernel_size=2, stride=2, padding=0)
    self.d2_C = DoubleConv(rev_channels[1], rev_channels[2])
    self.d1_T = nn.ConvTranspose2d(rev_channels[2], rev_channels[3], kernel_size=2, stride=2, padding=0)
    self.d1_C = DoubleConv(rev_channels[2], rev_channels[3])

    # Final convolution
    self.output = nn.Conv2d(rev_channels[3], out_channels, kernel_size=1)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x1 = self.e1(x)
    x = self.pool(x1)
    x2 = self.e2(x)
    x = self.pool(x2)
    x3 = self.e3(x)
    x = self.pool(x3)
    x4 = self.e4(x)
    x = self.pool(x4)
    x_bottom = self.bottom(x)
    x = self.d4_T(x_bottom)
    x = torch.cat((x4, x), dim=1)
    x = self.d4_C(x)
    x = self.d3_T(x)
    x = torch.cat((x3, x), dim=1)
    x = self.d3_C(x)
    x = self.d2_T(x)
    x = torch.cat((x2, x), dim=1)
    x = self.d2_C(x)
    x = self.d1_T(x)
    x = torch.cat((x1, x), dim=1)
    x = self.d1_C(x)
    x = self.output(x)
    return x

#################################################################################################################################
# Define data loader, load model, hyper-parameters and metrics
#################################################################################################################################
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model
model = UNet(3, 20, [64, 128, 256, 512]).to(device)
# Loss Function
n_classes = 20
loss_fn = nn.CrossEntropyLoss()
# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, min_lr= 0.00125, verbose=True)
lr = []

jaccard_micro = torchmetrics.JaccardIndex(task="multiclass", num_classes=20, average="micro").to(device)
jaccard_macro = torchmetrics.JaccardIndex(task="multiclass", num_classes=20, average="macro").to(device)
jaccard_weighted = torchmetrics.JaccardIndex(task="multiclass", num_classes=20, average="weighted").to(device)
jaccard_none = torchmetrics.JaccardIndex(task="multiclass", num_classes=20, average=None).to(device)

epochs = 50

len_train = len(train_loader)
len_val = len(val_loader)

train_loss_per_epoch_avg = []
train_jaccard_micro_per_epoch_avg = []
train_jaccard_macro_per_epoch_avg = []
train_jaccard_weighted_per_epoch_avg = []
train_jaccard_none_per_epoch_avg = []

val_loss_per_epoch_avg = []
val_jaccard_micro_per_epoch_avg = []
val_jaccard_macro_per_epoch_avg = []
val_jaccard_weighted_per_epoch_avg = []
val_jaccard_none_per_epoch_avg = []

best_val_loss = float('inf')

folder_path = "/content/drive/MyDrive/Cityscapes_data/256x512_4_batches/unet/"

#################################################################################################################################
# Training and validation loops
#################################################################################################################################
for e in range(epochs):
  train_loss_total = 0
  train_jaccard_micro_total=0
  train_jaccard_macro_total=0
  train_jaccard_weighted_total=0
  train_jaccard_none_total=0

  val_loss_total = 0
  val_jaccard_micro_total=0
  val_jaccard_macro_total=0
  val_jaccard_weighted_total=0
  val_jaccard_none_total=0

  model.train()

  print("Epoch: {}".format(e+1))

  # training
  for batch, (image, label) in enumerate(tqdm(train_loader)):
    optimizer.zero_grad()
    image = image.to(device)
    label = label.to(device)
    output = model(image)

    loss = loss_fn(output.to(torch.float32), torch.squeeze(label, 1).long())
    loss.backward()
    optimizer.step()
    train_loss_total += loss.item()

    x = torch.softmax(output, dim=1)
    index = torch.argmax(x, dim=1)
    y = torch.unsqueeze(index, dim=1)
    train_jaccard_micro_total += jaccard_micro(y, label)
    train_jaccard_macro_total += jaccard_macro(y, label)
    train_jaccard_weighted_total += jaccard_weighted(y, label)
    train_jaccard_none_total += jaccard_none(y, label)
    
  train_loss_per_epoch_avg.append(train_loss_total/len_train)
  print("Epoch: {}, Average train loss per epoch: {}".format(e+1, train_loss_per_epoch_avg[e]))
  train_jaccard_micro_per_epoch_avg.append(train_jaccard_micro_total/len_train)
  print("Epoch: {}, Average train jaccard_micro coeff per epoch: {}".format(e+1, train_jaccard_micro_per_epoch_avg[e]))
  train_jaccard_macro_per_epoch_avg.append(train_jaccard_macro_total/len_train)
  print("Epoch: {}, Average train jaccard_macro coeff per epoch: {}".format(e+1, train_jaccard_macro_per_epoch_avg[e]))
  train_jaccard_weighted_per_epoch_avg.append(train_jaccard_weighted_total/len_train)
  print("Epoch: {}, Average train jaccard_weighted coeff per epoch: {}".format(e+1, train_jaccard_weighted_per_epoch_avg[e]))
  train_jaccard_none_per_epoch_avg.append(train_jaccard_none_total/len_train)
  print("Epoch: {}, Average train jaccard_none coeff per epoch: {}".format(e+1, train_jaccard_none_per_epoch_avg[e]))

  train_loss_total = 0
  train_jaccard_micro_total = 0
  train_jaccard_macro_total = 0
  train_jaccard_weighted_total = 0
  train_jaccard_none_total = 0

  torch.save(model, folder_path + "city_unet.pth")
  file = open(folder_path + 'train_loss.txt','w')
  for item in train_loss_per_epoch_avg:
	  file.write(str(item)+"\n")
  file.close()

  torch.save(train_jaccard_micro_per_epoch_avg, folder_path + "train_jaccard_micro_per_epoch_avg.pth")
  torch.save(train_jaccard_macro_per_epoch_avg, folder_path + "train_jaccard_macro_per_epoch_avg.pth")
  torch.save(train_jaccard_weighted_per_epoch_avg, folder_path + "train_jaccard_weighted_per_epoch_avg.pth")
  torch.save(train_jaccard_none_per_epoch_avg, folder_path + "train_jaccard_none_per_epoch_avg.pth")


  # validating
  with torch.no_grad():
    model.eval()
    for batch, (image, label) in enumerate(tqdm(val_loader)):
      image = image.to(device)
      label = label.to(device)
      output = model(image)

      loss = loss_fn(output.to(torch.float32), torch.squeeze(label, 1).long())
      val_loss_total += loss.item()

      x = torch.softmax(output, dim=1)
      index = torch.argmax(x, dim=1)
      y = torch.unsqueeze(index, dim=1)
      val_jaccard_micro_total += jaccard_micro(y, label)
      val_jaccard_macro_total += jaccard_macro(y, label)
      val_jaccard_weighted_total += jaccard_weighted(y, label)
      val_jaccard_none_total += jaccard_none(y, label)

  scheduler.step(val_loss_total)
  print('epoch={}, learning rate={:.4f}'.format(e, optimizer.state_dict()['param_groups'][0]['lr']))
  lr.append(optimizer.state_dict()['param_groups'][0]['lr'])

  if val_loss_total < best_val_loss:
    torch.save(model.state_dict(), folder_path + 'best_weights.pth')
    best_val_loss = val_loss_total

  val_loss_per_epoch_avg.append(val_loss_total/len_val)
  print("Epoch: {}, Average val loss per epoch: {}".format(e+1, val_loss_per_epoch_avg[e]))
  val_jaccard_micro_per_epoch_avg.append(val_jaccard_micro_total/len_val)
  print("Epoch: {}, Average val jaccard_micro coeff per epoch: {}".format(e+1, val_jaccard_micro_per_epoch_avg[e]))
  val_jaccard_macro_per_epoch_avg.append(val_jaccard_macro_total/len_val)
  print("Epoch: {}, Average val jaccard_macro coeff per epoch: {}".format(e+1, val_jaccard_macro_per_epoch_avg[e]))
  val_jaccard_weighted_per_epoch_avg.append(val_jaccard_weighted_total/len_val)
  print("Epoch: {}, Average val jaccard_weighted coeff per epoch: {}".format(e+1, val_jaccard_weighted_per_epoch_avg[e]))
  val_jaccard_none_per_epoch_avg.append(val_jaccard_none_total/len_val)
  print("Epoch: {}, Average val jaccard_none coeff per epoch: {}".format(e+1, val_jaccard_none_per_epoch_avg[e]))

  val_loss_total = 0
  val_jaccard_micro_total = 0
  val_jaccard_macro_total = 0
  val_jaccard_weighted_total = 0
  val_jaccard_none_total = 0

  file = open(folder_path + 'val_loss.txt','w')
  for item in val_loss_per_epoch_avg:
	  file.write(str(item)+"\n")
  file.close()

  file = open(folder_path + 'lr.txt','w')
  for item in lr:
	  file.write(str(item)+"\n")
  file.close()

  torch.save(val_jaccard_micro_per_epoch_avg, folder_path + "val_jaccard_micro_per_epoch_avg.pth")
  torch.save(val_jaccard_macro_per_epoch_avg, folder_path + "/val_jaccard_macro_per_epoch_avg.pth")
  torch.save(val_jaccard_weighted_per_epoch_avg, folder_path + "/val_jaccard_weighted_per_epoch_avg.pth")
  torch.save(val_jaccard_none_per_epoch_avg, folder_path + "/val_jaccard_none_per_epoch_avg.pth")

Epoch: 1


100%|██████████| 170/170 [1:02:54<00:00, 22.20s/it]


Epoch: 1, Average train loss per epoch: 1.2029173440792982
Epoch: 1, Average train jaccard_micro coeff per epoch: 0.4798462986946106
Epoch: 1, Average train jaccard_macro coeff per epoch: 0.15607640147209167
Epoch: 1, Average train jaccard_weighted coeff per epoch: 0.47236499190330505
Epoch: 1, Average train jaccard_none coeff per epoch: tensor([3.6813e-01, 6.9308e-01, 6.7015e-02, 4.5904e-01, 9.0200e-06, 1.1032e-04,
        1.9667e-02, 3.4255e-06, 1.8047e-02, 5.4322e-01, 1.0138e-01, 6.3553e-01,
        2.5964e-04, 6.4108e-06, 2.1599e-01, 9.3503e-06, 6.7772e-06, 1.6211e-06,
        4.2816e-06, 9.1543e-06], device='cuda:0')


100%|██████████| 32/32 [10:43<00:00, 20.12s/it]


epoch=0, learning rate=0.0100
Epoch: 1, Average val loss per epoch: 1.2197827715426683
Epoch: 1, Average val jaccard_micro coeff per epoch: 0.484040230512619
Epoch: 1, Average val jaccard_macro coeff per epoch: 0.16365265846252441
Epoch: 1, Average val jaccard_weighted coeff per epoch: 0.49467676877975464
Epoch: 1, Average val jaccard_none coeff per epoch: tensor([0.3307, 0.7220, 0.1101, 0.5033, 0.0000, 0.0000, 0.0767, 0.0000, 0.0367,
        0.5511, 0.1009, 0.5625, 0.0173, 0.0000, 0.2617, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
Epoch: 2


...

epoch=48, learning rate=0.0013
Epoch: 49, Average val loss per epoch: 0.6262882398441434
Epoch: 49, Average val jaccard_micro coeff per epoch: 0.7518161535263062
Epoch: 49, Average val jaccard_macro coeff per epoch: 0.3948841989040375
Epoch: 49, Average val jaccard_weighted coeff per epoch: 0.7687046527862549
Epoch: 49, Average val jaccard_none coeff per epoch: tensor([0.6339, 0.8978, 0.5806, 0.7879, 0.1037, 0.1385, 0.3276, 0.1035, 0.2438,
        0.8483, 0.2966, 0.8277, 0.3663, 0.0800, 0.8256, 0.1443, 0.2330, 0.0481,
        0.0521, 0.3584], device='cuda:0')
Epoch: 50


100%|██████████| 170/170 [08:54<00:00,  3.14s/it]


Epoch: 50, Average train loss per epoch: 0.2225754415287691
Epoch: 50, Average train jaccard_micro coeff per epoch: 0.8796579241752625
Epoch: 50, Average train jaccard_macro coeff per epoch: 0.5635825395584106
Epoch: 50, Average train jaccard_weighted coeff per epoch: 0.8856905102729797
Epoch: 50, Average train jaccard_none coeff per epoch: tensor([0.8919, 0.9731, 0.8389, 0.8938, 0.3462, 0.4452, 0.4104, 0.1793, 0.3627,
        0.8906, 0.5621, 0.9025, 0.5660, 0.1886, 0.9107, 0.4404, 0.4268, 0.2748,
        0.2356, 0.5321], device='cuda:0')


100%|██████████| 32/32 [01:04<00:00,  2.01s/it]

epoch=49, learning rate=0.0013
Epoch: 50, Average val loss per epoch: 0.6233922271057963
Epoch: 50, Average val jaccard_micro coeff per epoch: 0.751009464263916
Epoch: 50, Average val jaccard_macro coeff per epoch: 0.39649248123168945
Epoch: 50, Average val jaccard_weighted coeff per epoch: 0.768438458442688
Epoch: 50, Average val jaccard_none coeff per epoch: tensor([0.6364, 0.8950, 0.5875, 0.7864, 0.1087, 0.1368, 0.3263, 0.0981, 0.2445,
        0.8524, 0.3031, 0.8148, 0.3686, 0.0811, 0.8205, 0.1373, 0.2572, 0.0539,
        0.0540, 0.3675], device='cuda:0')
